## SQL Queries

#### Task 1: How many stores does the business have and in which countries?

The Operations team would like to know which countries we currently operate in and which country now has the most stores. Perform a query on the database to get the information, it should return the following information:


In [ ]:
+----------+-----------------+
| country  | total_no_stores |
+----------+-----------------+
| GB       |             265 |
| DE       |             141 |
| US       |              34 |
+----------+-----------------+

In [ ]:
-- Subsequent SQL Query to extract this data

SELECT DISTINCT(country_code) as country,
	COUNT(country_code) as total_no_stores
FROM dim_store_details
GROUP BY country_code
ORDER BY total_no_stores DESC;

#### Task 2: Which locations currently have the most stores?

The business stakeholders would like to know which locations currently have the most stores.

They would like to close some stores before opening more in other locations.

Find out which locations have the most stores currently. The query should return the following:

In [ ]:
+-------------------+-----------------+
|     locality      | total_no_stores |
+-------------------+-----------------+
| Chapletown        |              14 |
| Belper            |              13 |
| Bushley           |              12 |
| Exeter            |              11 |
| High Wycombe      |              10 |
| Arbroath          |              10 |
| Rutherglen        |              10 |
+-------------------+-----------------+

In [ ]:
SELECT DISTINCT locality,
		COUNT(store_code) as total_no_stores
FROM dim_store_details
GROUP BY locality
ORDER BY COUNT(store_code) DESC
LIMIT 7;

#### Task 3: Which months produced the largest amount of sales?

Query the database to find out which months have produced the most sales. The query should return the following information:

In [ ]:
+-------------+-------+
| total_sales | month |
+-------------+-------+
|   673295.68 |     8 |
|   668041.45 |     1 |
|   657335.84 |    10 |
|   650321.43 |     5 |
|   645741.70 |     7 |
|   645463.00 |     3 |
+-------------+-------+

It appears you need to write a joining statement to join three tables to get date, product price & product quantity;
- dim_products (product_code & product_price columns)
- orders_table (product_code, product_quantity & date_uuid columns)
- dim_date_times (month & date_uuid columns)

The SQL Query:

In [ ]:
SELECT
	ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric,2) AS total_sales,
	dim_date_times.month
FROM
	orders_table
INNER JOIN
	dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
INNER JOIN
	dim_products ON dim_products.product_code = orders_table.product_code
GROUP BY dim_date_times.month
ORDER BY SUM(orders_table.product_quantity * dim_products.product_price) DESC
LIMIT 6;

In [ ]:
-- Working out/draft

-- First part

SELECT DISTINCT month
FROM dim_date_times
LIMIT 6;

-- orders_table.product_quantity * dim_products.product_price

-- Total orders per month

SELECT COUNT(orders_table.product_code) AS total_orders,
		dim_date_times.month
FROM
	orders_table
INNER JOIN
	dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
GROUP BY dim_date_times.month
ORDER BY COUNT(orders_table.product_code) DESC;

-- You need to multiply orders_table.product_quantity by dim_products.product_price

SELECT (orders_table.product_quantity * dim_products.product_price) AS total_product_sale,
		dim_date_times.month
FROM
	orders_table
INNER JOIN
	dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
INNER JOIN
	dim_products ON dim_products.product_code = orders_table.product_code
GROUP BY dim_date_times.month, orders_table.product_quantity, dim_products.product_price
ORDER BY dim_date_times.month DESC;



#### Task 4: How many sales are coming from online?

The company is looking to increase its online sales.
They want to know how many sales are happening online vs offline.
Calculate how many products were sold and the amount of sales made for online and offline purchases.
You should get the following information:

In [ ]:
+------------------+-------------------------+----------+
| numbers_of_sales | product_quantity_count  | location |
+------------------+-------------------------+----------+
|            26957 |                  107739 | Web      |
|            93166 |                  374047 | Offline  |
+------------------+-------------------------+----------+

You'll need to extract data from the following;
- dim_store_details (store_type & store_code columns)
    - you'll need store_type of Web and everything else grouped as one
- orders_table (product_quantity & card_number columns)
- a way to tally each individual payment completed linked to the above details
    - could that be with dim_card_details (and just card_number column?)

In [ ]:
orders_table will give you sale (product_code) and quantity (product_quantity)

-- Can we find out a way to see number of sales and product quantity per store_type?

To set location as Web or Offline, I believe you will need to create a new column
for the purpose of this task

SELECT
	COUNT(orders_table.product_code) AS number_of_sales,
	SUM(orders_table.product_quantity) AS product_quantity_count,
	dim_store_details.store_type
FROM
	orders_table
INNER JOIN
	dim_store_details ON dim_store_details.store_code = orders_table.store_code
GROUP BY
	dim_store_details.store_type;

-- This will group the sales information by the store type
-- Next you will need to create a new column to define the location Web, Offline

The SQL Query:

In [ ]:
SELECT
	COUNT(orders_table.product_code) AS number_of_sales,
	SUM(orders_table.product_quantity) AS product_quantity_count,
CASE
	WHEN dim_store_details.store_type = 'Web Portal' THEN 'Web'
	ELSE 'Offline'
END AS location
FROM
	orders_table
INNER JOIN
	dim_store_details ON dim_store_details.store_code = orders_table.store_code
GROUP BY
	CASE
		WHEN dim_store_details.store_type = 'Web Portal' THEN 'Web'
		ELSE 'Offline'
	END;

#### Task 5: What percentage of sales come through each type of store?

The sales team wants to know which of the different store types is generated the most revenue so they know where to focus.

Find out the total and percentage of sales coming from each of the different store types.

The query should return:


In [ ]:
+-------------+-------------+---------------------+
| store_type  | total_sales | percentage_total(%) |
+-------------+-------------+---------------------+
| Local       |  3440896.52 |               44.87 |
| Web portal  |  1726547.05 |               22.44 |
| Super Store |  1224293.65 |               15.63 |
| Mall Kiosk  |   698791.61 |                8.96 |
| Outlet      |   631804.81 |                8.10 |
+-------------+-------------+---------------------+

In [ ]:
-- Returns store_type;

SELECT store_type
FROM dim_store_details
GROUP BY store_type;

-- Returns number_of_sales;
SELECT COUNT(store_code)
FROM orders_table;


-- This returns everything, apart from the percentage_total column
SELECT
	dim_store_details.store_type,
	ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric,2) AS total_sales
	(SUM(orders_table.product_quantity * dim_products.product_price)) AS percentage_total(%)
FROM
	orders_table
INNER JOIN
	dim_store_details ON orders_table.store_code = dim_store_details.store_code
INNER JOIN
	dim_products ON dim_products.product_code = orders_table.product_code
GROUP BY dim_store_details.store_type
ORDER BY ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric,2) DESC;

To complete this, I think you might need to use a CTE or a subquery to calculate the total_sales

The SQL Query:

In [ ]:
WITH TotalSales AS (
    SELECT
        dim_store_details.store_type,
        SUM(orders_table.product_quantity * dim_products.product_price) AS total_sales
    FROM
        orders_table
    INNER JOIN
        dim_store_details ON orders_table.store_code = dim_store_details.store_code
    INNER JOIN
        dim_products ON dim_products.product_code = orders_table.product_code
    GROUP BY dim_store_details.store_type
), TotalSalesSum AS (
    SELECT SUM(total_sales) AS total_sum
    FROM TotalSales
)

SELECT
    TS.store_type,
    ROUND(TS.total_sales::numeric, 2) AS total_sales,
    ROUND(((TS.total_sales / TSS.total_sum) * 100)::numeric, 2) AS percentage_total

FROM
    TotalSales TS, TotalSalesSum TSS
ORDER BY
    TS.total_sales DESC;

#### Task 6: Which month in each year produced the highest cost of sales?

The company stakeholders want assurances that the company has been doing well recently.

Find which months in which years have had the most sales historically.

The query should return the following information:

In [ ]:
+-------------+------+-------+
| total_sales | year | month |
+-------------+------+-------+
|    27936.77 | 1994 |     3 |
|    27356.14 | 2019 |     1 |
|    27091.67 | 2009 |     8 |
|    26679.98 | 1997 |    11 |
|    26310.97 | 2018 |    12 |
|    26277.72 | 2019 |     8 |
|    26236.67 | 2017 |     9 |
|    25798.12 | 2010 |     5 |
|    25648.29 | 1996 |     8 |
|    25614.54 | 2000 |     1 |
+-------------+------+-------+

SQL Query:

In [ ]:
SELECT
		ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric, 2) AS total_sales,
		dim_date_times.year,
		dim_date_times.month
    FROM
        orders_table
    INNER JOIN
        dim_date_times ON orders_table.date_uuid = dim_date_times.date_uuid
    INNER JOIN
        dim_products ON dim_products.product_code = orders_table.product_code
    GROUP BY dim_date_times.year, dim_date_times.month
	ORDER BY total_sales DESC
	LIMIT 10;

#### Task 7: What is our staff headcount?

The operations team would like to know the overall staff numbers in each location around the world.
Perform a query to determine the staff numbers in each of the countries the company sells in.

The query should return the values:

In [ ]:
+---------------------+--------------+
| total_staff_numbers | country_code |
+---------------------+--------------+
|               13307 | GB           |
|                6123 | DE           |
|                1384 | US           |
+---------------------+--------------+

The SQL Query:

In [ ]:
SELECT
	SUM(staff_numbers) AS total_staff_numbers,
	country_code
FROM
	dim_store_details
GROUP BY
	country_code
ORDER BY
	total_staff_numbers DESC;

#### Task 8: Which German store type is selling the most?

The sales team is looking to expand their territory in Germany.
Determine which type of store is generating the most sales in Germany.

The query will return:

In [ ]:
+--------------+-------------+--------------+
| total_sales  | store_type  | country_code |
+--------------+-------------+--------------+
|   198373.57  | Outlet      | DE           |
|   247634.20  | Mall Kiosk  | DE           |
|   384625.03  | Super Store | DE           |
|  1109909.59  | Local       | DE           |
+--------------+-------------+--------------+

The SQL Query:

In [ ]:
SELECT
	ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric,2) AS total_sales,
	dim_store_details.store_type,
	dim_store_details.country_code
FROM
	orders_table
INNER JOIN
	dim_store_details ON orders_table.store_code = dim_store_details.store_code
INNER JOIN
	dim_products ON dim_products.product_code = orders_table.product_code
WHERE
	dim_store_details.country_code = 'DE'
GROUP BY
	dim_store_details.store_type, dim_store_details.country_code
ORDER BY ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric,2);

#### Task 9: How quickly is the company making sales?

Sales would like the get an accurate metric for how quickly the company is making sales.

Determine the average time taken between each sale grouped by year, the query should return the following information.

Hint: You will need the SQL command LEAD.

In [ ]:
 +------+-------------------------------------------------------+
 | year |                           actual_time_taken           |
 +------+-------------------------------------------------------+
 | 2013 | "hours": 2, "minutes": 17, "seconds": 12, "millise... |
 | 1993 | "hours": 2, "minutes": 15, "seconds": 35, "millise... |
 | 2002 | "hours": 2, "minutes": 13, "seconds": 50, "millise... | 
 | 2022 | "hours": 2, "minutes": 13, "seconds": 6,  "millise... |
 | 2008 | "hours": 2, "minutes": 13, "seconds": 2,  "millise... |
 +------+-------------------------------------------------------+

In [ ]:
-- What columns do you need?
dim_date_times.year
COUNT(dim_date_times.date_uuid)
GROUP BY year


In [ ]:
WITH TimeDifferences AS (
    SELECT
        dim_date_times.year,
        EXTRACT(EPOCH FROM (
            LEAD(
                CAST(
                    dim_date_times.year || '-' || 
                    dim_date_times.month || '-' || 
                    dim_date_times.day || ' ' || 
                    dim_date_times.timestamp
                AS timestamp)
            ) OVER (PARTITION BY dim_date_times.year ORDER BY dim_date_times.year, dim_date_times.month, dim_date_times.day, dim_date_times.timestamp) 
            - CAST(
                dim_date_times.year || '-' || 
                dim_date_times.month || '-' || 
                dim_date_times.day || ' ' || 
                dim_date_times.timestamp
            AS timestamp)
        )) AS time_diff
    FROM
        orders_table
    INNER JOIN
        dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
)

SELECT
    year,
    '"' || 'hours' || '": ' || FLOOR(AVG(time_diff) / 3600) || ', ' ||
    '"' || 'minutes' || '": ' || FLOOR((AVG(time_diff) % 3600) / 60) || ', ' ||
    '"' || 'seconds' || '": ' || FLOOR(AVG(time_diff) % 60) || ', ' ||
    '"' || 'milliseconds' || '": ' || ROUND((AVG(time_diff) - FLOOR(AVG(time_diff))) * 1000) AS actual_time_taken
FROM 
    TimeDifferences
WHERE 
    time_diff IS NOT NULL
GROUP BY 
    year
ORDER BY 
    actual_time_taken DESC
LIMIT 5;